# **ChatterBot Implementation of Telegram Data**

## **Installing prereqs**

In [14]:
!pip install chatterbot
!pip install spacy
!pip install sqlalchemy
!pip install PyYAML
!pip install chatterbot_corpus
!pip install en
!pip install emoji

  Using cached https://files.pythonhosted.org/packages/6c/0e/dac0d82f34f86bf509cf5ef3e2dfc5aa7d444bd843a2330ceb7d854f84f2/ChatterBot-1.0.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/f3/554271be8ff46471586d164bfbb6999364ba30ca5a0045e2a86da5f3b2c5/spacy-2.1.9-cp37-cp37m-manylinux1_x86_64.whl
^C
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import re
import emoji

In [25]:
!spacy download en

     |████████████████████████████████| 12.0MB 1.6MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp37-none-any.whl size=12011740 sha256=f642172442fb0375360bcf4ecef7956912d0ad827f0b080eb97b9b01ec08a55e
  Stored in directory: /tmp/pip-ephem-wheel-cache-nj2ufvvw/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/venv/lib/python3.7/site-packages/en_core_web_sm -->
/opt/venv/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


## **Training Chatterbot on Original English Corpus**

In [26]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer


chatbot = ChatBot('Steven')

# Create a new trainer for the chatbot
trainer = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the english corpus
trainer.train("chatterbot.corpus.english")

/home/jovyan/work/chatterbot/corpus.py:38: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(data_file)
Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psyc

In [27]:
# Get a response to an input statement
chatbot.get_response("Hello, how are you today?")

<Statement text:but I need to finish it ASAP cause I will have classes and then assignment can you help me brainstorm ideas when youre free later>

## **Pre-processing my Telegram Data**

In [0]:
with open('telegram.txt', 'r') as telegram:
    telegram_data = telegram.readlines()
# checking the messages
telegram_data[-10:]

In [0]:
def remove_emoji(text, remove_components=False):
    '''
    Script to remove emojis from string, retrieved from:
    https://stackoverflow.com/a/51785357
    '''
    cleaned = emoji.get_emoji_regexp().sub(u'', text)
    if remove_components:
        cleaned = emoji_components.sub(u'', cleaned)
    return cleaned

def strip_text(text):
    '''
    Process the text further
    '''
    noEmojiMessage = remove_emoji(text)
    # Remove any links in the message
    noLinksMessage = re.sub(r'^https?:\/\/.*[\r\n]*', '', noEmojiMessage, flags=re.MULTILINE)
    # Remove any email addresses in the message
    processed_line = noLinksMessage.replace('\S*@\S*\s?',' ')
    first_colon = processed_line.find(': ') + 1
    just_message = processed_line[first_colon:].strip()
    sender_name = processed_line[:first_colon]
    return processed_line, first_colon, just_message, sender_name

def progressBar(current, total, barLength = 20):
    '''
    Standard progress bar algorithm retrieved from https://stackoverflow.com/a/37630397
    '''
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))

    print('Processing the data: [%s%s] %d %%' % (arrow, spaces, percent), end='\r')


def get_Messages(message_data):
    '''
    Main function to process the data:
    - message_data: the txt file with all the extracted messages
    '''
    responseDictionary = dict()
    myMessage, otherPersonsMessage, currentSpeaker = "","",""
    forbidden_list = ["[[Photo", "[[Video", "[[Voice", "[[GIF", "[[Webpage]]", 
                          "[[?messageMediaUnsupported]]", "{{FWD", "{{GIF", "[[Geo"
                          "[[Document", ">>incoming call", ">>outgoing call", 
                          ">>missed call", ">>canceled call"]
    startMessageIndex = 0
    process_count = 0
    for idx, line in enumerate(message_data):
        processed_line, first_colon, just_message, sender_name = strip_text(line)
        if len(just_message) < 3:
            continue
        elif bool([ele for ele in forbidden_list if(ele in just_message)]) == True:
            continue
        elif "[[" and "Sticker" in just_message:
            continue
        else:
            # if the current message is from me
            if "Steven Tey" in sender_name: 
                if not myMessage:
                    # Get the first message I sent 
                    # (if I send multiple messages in a row)
                    startMessageIndex = idx - 1
                # add current message to myMessage cache
                myMessage = myMessage + ' ' + just_message
            elif myMessage: # if I'm not the one speaking 
                for counter in range(startMessageIndex, 0, -1):
                    currentLine = message_data[counter]
                    processed_line_other, first_colon_other, just_message_other, sender_name_other = strip_text(currentLine)
                    if len(just_message_other) < 3:
                        continue
                    elif bool([ele for ele in forbidden_list if(ele in just_message)]) ==  True:
                        continue
                    elif "[[" and "Sticker" in just_message_other:
                        continue
                    else:
                        if not currentSpeaker:
                            # If the currentSpeaker variable is empty, 
                            # assign Anzhelika as current speaker
                            if "Anzhelika" in sender_name_other:
                                currentSpeaker = "Anzhelika"
                        # Once there is a current speaker and the 
                        # otherPersonsMessage variable is not empty 
                        elif ("Anzhelika" not in sender_name_other and otherPersonsMessage):
                            # A different person started speaking, so now I know that the
                            # first person's message is done
                            responseDictionary[otherPersonsMessage] = myMessage
                            break
                        # But if those two conditions don't hold
                        # add current message to otherPersonsMessage cache
                        otherPersonsMessage = just_message_other + ' ' + otherPersonsMessage
                # clear caches at the end of the loop
                myMessage, otherPersonsMessage, currentSpeaker = "","",""  
        process_count += 1
        progressBar(process_count, len(message_data), barLength = 30)
    return responseDictionary       

In [0]:
responseDict = get_Messages(telegram_data)

In [0]:
with open("telegram.yaml", "w") as f:
    for key in responseDict:
        # Write two lines to the file.
        f.write("\n- - ")
        f.write(key)
        f.write("\n  - ")
        f.write(responseDict[key])

## **Training Chatterbot on A Subset of my Telegram Data**

In [0]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

chatbot = ChatBot('Steven')

# Create a new trainer for the chatbot
trainer = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the english corpus
trainer.train("telegram-subset")

In [0]:
# Get a response to an input statement
chatbot.get_response("I love you baby")

## **Training Chatterbot on A Reversed Subset of my Telegram Data**

By "reversed", I mean that the prompt and response pairs were reversed to allow me to see what Anzhelika will reply when I say something.

In [0]:
chatbot_anzhelika = ChatBot('Anzhelika')

# Create a new trainer for the chatbot
trainer_anzhelika = ChatterBotCorpusTrainer(chatbot_anzhelika)

# Train the chatbot based on the english corpus
trainer_anzhelika.train("telegram-subset-reversed")

In [0]:
# Get a response to an input statement
chatbot_anzhelika.get_response("I love you with all my heart")

## **Training Chatterbot on My Full Telegram Data**

In [28]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

telegram_chatbot = ChatBot('Steven Tey')

# Create a new trainer for the chatbot
telegram_trainer = ChatterBotCorpusTrainer(telegram_chatbot)

# Train the chatbot based on the english corpus
telegram_trainer.train("telegram")

Training telegram.yml: [####################] 100%


In [34]:
# Get a response to an input statement
telegram_chatbot.get_response("Yes baby?")

<Statement text:Have an amazing day with your friends!! Good later️>